# Explore and cluster the neighborhoods in Toronto

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests
import geocoder
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

### Load Data from File

In [5]:
toronto_df = pd.read_csv('toronto_postal_codes_ver2.csv')
toronto_df.head()

PostalCode      Borough                            Neighborhood  Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.81139   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.78574   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.76575   
3        M1G  Scarborough                                  Woburn  43.76812   
4        M1H  Scarborough                               Cedarbrae  43.76944   

   Longitude  
0  -79.19662  
1  -79.15875  
2  -79.17470  
3  -79.21761  
4  -79.23892

### Create a map of Toronto with neighborhoods superimposed on top

In [11]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 4,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#87cefa',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_toronto)
map_toronto

In [13]:
# Foursquare API
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # Put Your Client Id
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # Put You Client Secret 
VERSION = '20180604'
LIMIT = 30

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_data = toronto_df[toronto_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

PostalCode                 Borough                    Neighborhood  \
0        M4E            East Toronto                     The Beaches   
1        M4J  East York/East Toronto              The Danforth  East   
2        M4K            East Toronto    The Danforth West, Riverdale   
3        M4L            East Toronto  India Bazaar, The Beaches West   
4        M4M            East Toronto                 Studio District   

   Latitude  Longitude  
0  43.67709  -79.29547  
1  43.68811  -79.33418  
2  43.68375  -79.35512  
3  43.66797  -79.31467  
4  43.66213  -79.33497

In [16]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

### Exploring Neighbourhood in Toronto

In [19]:
df = toronto_data
toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [20]:
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         The Beaches               43.67709               -79.29547   
1         The Beaches               43.67709               -79.29547   
2         The Beaches               43.67709               -79.29547   
3         The Beaches               43.67709               -79.29547   
4  The Danforth  East               43.68811               -79.33418   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                  Mos Eisley Cantina       43.687242       -79.328910   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3       Neighborhood  
4        Music Venue

In [21]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            30   
Brockton, Parkdale Village, Exhibition Place                           30   
CN Tower, King and Spadina, Railway Lands, Harb...                     30   
Central Bay Street                                                     30   
Christie                                                               11   
Church and Wellesley                                                   30   
Commerce Court, Victoria Hotel                                         30   
Davisville                                                             29   
Davisville North                                                        6   
Dufferin, Dovercourt Village                                           17   
Enclave of M4L                                                         30   
Enclave of M5E                                                         30   
First Canadian Place, Underground city                                 30   
Forest Hill North & West                                                2   
Garden District, Ryerson                                               30   
Harbourfront East, Union Station, Toronto Islands                      30   
High Park, The Junction South                                           2   
India Bazaar, The Beaches West                                         20   
Kensington Market, Chinatown, Grange Park                              30   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               30   
Moore Park, Summerhill East                                             4   
North Toronto West                                                      3   
Parkdale, Roncesvalles                                                 30   
Regent Park, Harbourfront                                              21   
Richmond, Adelaide, King                                               30   
Rosedale                                                                5   
Roselawn                                                                2   
Runnymede, Swansea                                                     30   
St. James Town                                                         30   
St. James Town, Cabbagetown                                            30   
Studio District                                                        30   
Summerhill West, Rathnelly, South Hill, Forest ...                      7   
The Annex, North Midtown, Yorkville                                    24   
The Beaches                                                             4   
The Danforth  East                                                      3   
The Danforth West, Riverdale                                            7   
Toronto Dominion Centre, Design Exchange                               30   
University of Toronto, Harbord                                         30   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             30   
Brockton, Parkdale Village, Exhibition Place                            30   
CN Tower, King and Spadina, Railway Lands, Harb...                      30   
Central Bay Street                                                      30   
Christie                                                                11   
Church and Wellesley                                                    30   
Commerce Court, Victoria Hotel                                          30   
Davisville                                                              29   
Davisville North                                                         6   
Dufferin, Dovercourt Village        

### Analyze Each Borough Neighborhood

In [22]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0            0                    0             0         0            0   
1            0                    0             0         0            0   
2            0                    0             0         0            0   
3            0                    0             0         0            0   
4            0                    0             0         0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   BBQ Joint  ...  Theater  Theme Restaurant  Toy / Game Store  Trail  \
0          0  ...        0                 0                 0      1   
1          0  ...        0                 0                 0      0   
2          0  ...        0                 0                 0      0   
3          0  ...        0                 0                 0      0   
4          0  ...        0                 0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  
0                      0         0          0  
1                      0         0          0  
2                      0         0          0  
3                      0         0          0  
4                      0         0          0  

[5 rows x 176 columns]

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                                        Berczy Park     0.000000   
1       Brockton, Parkdale Village, Exhibition Place     0.000000   
2  CN Tower, King and Spadina, Railway Lands, Har...     0.033333   
3                                 Central Bay Street     0.000000   
4                                           Christie     0.000000   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                  0.0           0.0       0.0     0.033333    0.000000   
1                  0.0           0.0       0.0     0.000000    0.000000   
2                  0.0           0.0       0.0     0.000000    0.000000   
3                  0.0           0.0       0.0     0.000000    0.033333   
4                  0.0           0.0       0.0     0.000000    0.000000   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  ...  Theater  \
0             0.000000          0.000000            0.000000  ...      0.0   
1             0.033333          0.000000            0.000000  ...      0.0   
2             0.000000          0.033333            0.000000  ...      0.0   
3             0.000000          0.000000            0.000000  ...      0.0   
4             0.000000          0.000000            0.090909  ...      0.0   

   Theme Restaurant  Toy / Game Store  Trail  Train Station  \
0               0.0               0.0    0.0            0.0   
1               0.0               0.0    0.0            0.0   
2               0.0               0.0    0.0            0.0   
3               0.0               0.0    0.0            0.0   
4               0.0               0.0    0.0            0.0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                       0.033333               0.0                    0.0   
1                       0.033333               0.0                    0.0   
2                       0.000000               0.0                    0.0   
3                       0.000000               0.0                    0.0   
4                       0.000000               0.0                    0.0   

   Wine Bar  Wine Shop  
0       0.0        0.0  
1       0.0        0.0  
2       0.0        0.0  
3       0.0        0.0  
4       0.0        0.0  

[5 rows x 176 columns]

In [24]:
num_top_venues = 5
for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1        Cocktail Bar  0.07
2            Beer Bar  0.07
3      Farmers Market  0.07
4              Bakery  0.07


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0               Gift Shop  0.07
1  Furniture / Home Store  0.07
2              Restaurant  0.07
3             Coffee Shop  0.07
4      Italian Restaurant  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Italian Restaurant  0.10
1                  Park  0.07
2  Gym / Fitness Center  0.07
3            Restaurant  0.07
4  Caribbean Restaurant  0.03


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.10
1           Plaza  0.07
2  Clothing Store  0.07
3      Poke Place  0.03
4             Spa  0.03


----Christie----
                venue  freq
0                Café  0.27
1       Grocery Stor

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park    Seafood Restaurant   
1       Brockton, Parkdale Village, Exhibition Place             Gift Shop   
2  CN Tower, King and Spadina, Railway Lands, Har...    Italian Restaurant   
3                                 Central Bay Street           Coffee Shop   
4                                           Christie                  Café   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0            Cocktail Bar              Beer Bar        Farmers Market   
1  Furniture / Home Store            Restaurant           Coffee Shop   
2                    Park  Gym / Fitness Center            Restaurant   
3                   Plaza        Clothing Store            Poke Place   
4           Grocery Store    Athletics & Sports           Candy Store   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                Bakery        Breakfast Spot                Bistro   
1    Italian Restaurant                Bakery                 Hotel   
2  Caribbean Restaurant                 Diner                   Spa   
3                   Spa       Bubble Tea Shop                  Park   
4    Italian Restaurant            Baby Store           Coffee Shop   

  8th Most Common Venue    9th Most Common Venue     10th Most Common Venue  
0            Restaurant             Concert Hall               Liquor Store  
1                  Café                      Spa             Sandwich Place  
2                Market                     Café         Seafood Restaurant  
3         Shopping Mall           Sandwich Place  Middle Eastern Restaurant  
4            Playground  New American Restaurant                Opera House

### Clustering Neighborhoods

In [29]:
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, 
  verbose=True)

In [30]:
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[0 0 0 0 6 0 6 0 0 0]
39


In [31]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

PostalCode                 Borough                    Neighborhood  \
0        M4E            East Toronto                     The Beaches   
1        M4J  East York/East Toronto              The Danforth  East   
2        M4K            East Toronto    The Danforth West, Riverdale   
3        M4L            East Toronto  India Bazaar, The Beaches West   
4        M4M            East Toronto                 Studio District   

   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.67709  -79.29547               5     Health Food Store   
1  43.68811  -79.33418               8                  Park   
2  43.68375  -79.35512               0         Grocery Store   
3  43.66797  -79.31467               0        Sandwich Place   
4  43.66213  -79.33497               0    Italian Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                 Trail                   Pub           Yoga Studio   
1           Music Venue          Intersection           Yoga Studio   
2                  Park              Bus Line      Business Service   
3    Italian Restaurant  Fast Food Restaurant         Burrito Place   
4               Brewery                 Diner           Coffee Shop   

       5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0         Miscellaneous Shop                Lounge      Malay Restaurant   
1  Middle Eastern Restaurant                Lounge      Malay Restaurant   
2             Discount Store        Ice Cream Shop          Intersection   
3          Fish & Chips Shop          Liquor Store            Restaurant   
4        Arts & Crafts Store   American Restaurant     French Restaurant   

       8th Most Common Venue    9th Most Common Venue  \
0                     Market      Martial Arts School   
1                     Market      Martial Arts School   
2         Miscellaneous Shop      Martial Arts School   
3          Food & Drink Shop               Steakhouse   
4  Latin American Restaurant  Comfort Food Restaurant   

     10th Most Common Venue  
0  Mediterranean Restaurant  
1  Mediterranean Restaurant  
2  Mediterranean Restaurant  
3          Sushi Restaurant  
4          Sushi Restaurant

In [34]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters